In [1]:
import numpy as np
import pandas as pd
import h5py
import scipy
import scanpy as sc
import anndata 
import os 
import sys

path='/home/ubuntu0/scBasset'

# from scbasset.utils import *

# plotting functions
import seaborn as sns
import matplotlib.pyplot as plt

import os

In [11]:
data_path = path+'/data/multiome_pbmc/'

h5_file = data_path + 'pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5'
bed_file = data_path + 'pbmc_granulocyte_sorted_3k_atac_peaks.bed'

In [12]:
peak = pd.read_csv(bed_file, sep='\t', names=['chr','start','end'])
ad = sc.read_10x_h5(h5_file, gex_only=False)

/home/ubuntu0/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [13]:
ad

AnnData object with n_obs × n_vars = 2714 × 193208
    var: 'gene_ids', 'feature_types', 'genome'

### filtering

In [14]:
ad_rna = ad[:, ad.var['feature_types']=='Gene Expression']
ad_atac = ad[:, ad.var['feature_types']=='Peaks']
ad_atac.var['chr'] = peak['chr'].values
ad_atac.var['start'] = peak['start'].values
ad_atac.var['end'] = peak['end'].values

# basic stats
sc.pp.filter_cells(ad_rna, min_genes=0)
sc.pp.filter_genes(ad_rna, min_cells=0)
sc.pp.filter_cells(ad_atac, min_genes=0)
sc.pp.filter_genes(ad_atac, min_cells=0)

# a gene need to be expressed in 5% cells
# a peak need to be accessible in 5% cells
thres = int(ad.shape[0]*0.05)
ad_rna = ad_rna[:, ad_rna.var['n_cells']>thres]
ad_atac = ad_atac[:, ad_atac.var['n_cells']>thres]

/home/ubuntu0/miniconda3/envs/scbasset/lib/python3.7/site-packages/ipykernel_launcher.py:3: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu0/miniconda3/envs/scbasset/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


In [15]:
ad_rna,ad_atac

(View of AnnData object with n_obs × n_vars = 2714 × 8652
     obs: 'n_genes'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 View of AnnData object with n_obs × n_vars = 2714 × 27157
     obs: 'n_genes'
     var: 'gene_ids', 'feature_types', 'genome', 'chr', 'start', 'end', 'n_cells')

In [9]:
ad_rna.write('rna_ad.h5ad')

### save h5ad

In [10]:
chrs = ['chr'+str(i) for i in range(1,23)] + ['chrX', 'chrY']
print(chrs)
ad_atac = ad_atac[:, ad_atac.var['chr'].isin(chrs)]
ad_atac.write('atac_ad.h5ad')

['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY']


/home/ubuntu0/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/ubuntu0/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


In [2]:
import anndata as ad
ad_atac2=ad.read_h5ad(path+'/atac_ad.h5ad')
ad_rna2=ad.read_h5ad(path+ '/rna_ad.h5ad')
ad_atac2, ad_rna2

(AnnData object with n_obs × n_vars = 2714 × 27150
     obs: 'n_genes'
     var: 'gene_ids', 'feature_types', 'genome', 'chr', 'start', 'end', 'n_cells',
 AnnData object with n_obs × n_vars = 2714 × 8652
     obs: 'n_genes'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells')

In [3]:
atac_file=path+'/atac_ad.h5ad'
process_file=path+'/processed'

## preprocess

In [4]:
def call_preprocess(file,process):   
    from preprocess import main
    main(file,'/home/ubuntu0/scBasset/hg38.fa',process)  

def call_train(pth,epoch,bottle_neck,output,trained_model):
    from train import main
    main(pth,epoch,bottle_neck,output,trained_model)


In [11]:
# !python bin/scbasset_preprocess.py --ad_file atac_ad.h5ad --input_fasta hg38.fa
call_preprocess(atac_file,process_file)

Namespace(ad_file='/home/ubuntu0/scBasset/atac_ad.h5ad', input_fasta='/home/ubuntu0/scBasset/hg38.fa', out_path='/home/ubuntu0/scBasset/processed')
successful writing h5ad file.
successful writing bed file.
successful writing split file.
successful writing sparse m.
process 0 peaks takes 0.7 s
process 1000 peaks takes 1.1 s
process 2000 peaks takes 1.5 s
process 3000 peaks takes 2.0 s
process 4000 peaks takes 2.4 s
process 5000 peaks takes 2.8 s
process 6000 peaks takes 3.2 s
process 7000 peaks takes 3.5 s
process 8000 peaks takes 3.9 s
process 9000 peaks takes 4.2 s
process 10000 peaks takes 4.6 s
process 11000 peaks takes 4.8 s
process 12000 peaks takes 5.1 s
process 13000 peaks takes 5.4 s
process 14000 peaks takes 5.9 s
process 15000 peaks takes 6.3 s
process 16000 peaks takes 6.6 s
process 17000 peaks takes 7.0 s
process 18000 peaks takes 7.3 s
process 19000 peaks takes 7.6 s
process 20000 peaks takes 8.0 s
process 21000 peaks takes 8.3 s
process 22000 peaks takes 8.7 s
process 23

## Training

In [5]:
call_train(process_file,200,32,path+'/output',None)

2024-02-02 21:29:27.915267: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-02 21:29:28.676214: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-02 21:29:29.005144: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 21:29:30.322312: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

cpu memory used: 0.9GB.


2024-02-02 21:29:32.027870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-02 21:29:32.094678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-02 21:29:32.094735: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-02 21:29:32.100363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the ap

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1344, 4)]    0           []                               
                                                                                                  
 stochastic_reverse_complement   ((None, 1344, 4),   0           ['sequence[0][0]']               
 (StochasticReverseComplement)   ())                                                              
                                                                                                  
 stochastic_shift (StochasticSh  (None, 1344, 4)     0           ['stochastic_reverse_complement[0
 ift)                                                            ][0]']                           
                                                                                              

2024-02-02 21:29:38.921244: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2024-02-02 21:29:42.428329: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-02-02 21:29:42.428558: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-02-02 21:29:42.428836: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-02-02 21:29:46.276195: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


191/191 [==============================] - 43s 154ms/step - loss: 0.5033 - auc: 0.5895 - auc_1: 0.2608 - val_loss: 0.4795 - val_auc: 0.6197 - val_auc_1: 0.2904
Epoch 2/200
191/191 [==============================] - 28s 143ms/step - loss: 0.4779 - auc: 0.6177 - auc_1: 0.2919 - val_loss: 0.4886 - val_auc: 0.6199 - val_auc_1: 0.2924
Epoch 3/200
191/191 [==============================] - 28s 143ms/step - loss: 0.4753 - auc: 0.6247 - auc_1: 0.3005 - val_loss: 0.4823 - val_auc: 0.6211 - val_auc_1: 0.3028
Epoch 4/200
191/191 [==============================] - 28s 143ms/step - loss: 0.4723 - auc: 0.6342 - auc_1: 0.3099 - val_loss: 0.4764 - val_auc: 0.6360 - val_auc_1: 0.3064
Epoch 5/200
191/191 [==============================] - 28s 143ms/step - loss: 0.4712 - auc: 0.6373 - auc_1: 0.3137 - val_loss: 0.4971 - val_auc: 0.6305 - val_auc_1: 0.3061
Epoch 6/200
191/191 [==============================] - 28s 143ms/step - loss: 0.4701 - auc: 0.6419 - auc_1: 0.3156 - val_loss: 0.5066 - val_auc: 0.6148 